In [0]:
DECLARE OR REPLACE VARIABLE var_as_of_date DATE;
SET VAR var_as_of_date = '${as_of_date}' :: DATE;

In [0]:
CREATE OR REPLACE TEMP VIEW filtered_sales_records_temp_v AS
SELECT *,
CURRENT_TIMESTAMP() AS load_timestamp, CURRENT_USER() AS loaded_by
FROM default.sales_records
WHERE order_date <= var_as_of_date

In [0]:
MERGE INTO default.sales_brnz t
USING filtered_sales_records_temp_v s
ON s.order_id = t.order_id
WHEN MATCHED 
THEN UPDATE 
SET t.load_timestamp = s.load_timestamp,
  t.loaded_by = s.loaded_by
WHEN NOT MATCHED THEN INSERT (region, country, item_type, sales_channel, order_priority, order_date, order_id, ship_date, units_sold, unit_price, unit_cost, total_revenue, total_cost, total_profit,  load_timestamp, loaded_by)
VALUES (s.region, s.country, s.item_type, s.sales_channel, s.order_priority, s.order_date, s.order_id, s.ship_date, s.units_sold, s.unit_price, s.unit_cost, s.total_revenue, s.total_cost, s.total_profit, s.load_timestamp, s.loaded_by)